In [2]:
import numpy as np
import torch 
import sys
sys.path.append('/Users/nezamjazayeri/Documents/neu/cs5180/proj/src')

from networks.mlp import MLP
from utils.parse_config import load_config

In [42]:
config = load_config('config/default.yaml')
num_players = config['env_args']['num_players']


def generate(t, winner=False):

    if t == 0:
        return [0, num_players, 0, 0, 0]
    res = [0, 0, 0, 0, 0]
    res[0] = t
    res[1] = max(num_players - (t * 4), 2)
    res[2] = max(1, max(num_players - ((t - 1) * 4), 2) - 1)
    res[3] = 0
    if winner == True:
        res[3] = 1
    res[4] = 1
    return res

STATES = [
    generate(0),
    generate(1),
    generate(2),
    generate(3),
    generate(4)
]

STATES_v2 = [
    generate(0, winner=True),
    generate(1, winner=True),
    generate(2, winner=True),
    generate(3, winner=True),
    generate(4, winner=True)
]

state_size = 5
action_size = 2

model = MLP(state_size, action_size, config)
load = config['total_train_t']
path = f"/Users/nezamjazayeri/Documents/neu/cs5180/proj/res/models/PPO/20240428/{load}/policy.pt"
model.load_state_dict(torch.load(path))


<All keys matched successfully>

In [43]:
import torch.nn.functional as F
def get_policy(state):
    
    with torch.no_grad():  # No gradient computation for action selection
        state = torch.FloatTensor(state).unsqueeze(0)  # Convert state to tensor and add batch dimension
        logits = model(state)
        probs = F.softmax(logits, dim=-1)
    print(f'@ round {int(state[0][0].item()) + 1}:',
          f'q0= {probs[0][0].item()},',
          f'q1= {probs[0][1].item()}')

In [44]:
for state in STATES:
    get_policy(state)

@ round 1: q0= 0.17418237030506134, q1= 0.8258176445960999
@ round 2: q0= 0.2386985570192337, q1= 0.7613014578819275
@ round 3: q0= 0.24906538426876068, q1= 0.7509346008300781
@ round 4: q0= 0.28186753392219543, q1= 0.7181324362754822
@ round 5: q0= 0.3171650767326355, q1= 0.6828349232673645
